### 역발상 종목 선별

In [1]:
import sys
import pandas as pd
from tqdm import tqdm
sys.path.append('../')
from stock_screener.data_reader import *

In [3]:
# 최근 거래일자 획득
date = get_recent_biz_day()

#해당일의 종목별 지표 정보 획득
per_pbr_divdend_data = get_per_pbr_dividend2(date)

In [7]:
# 종목의 업종분류 획득 (WICS)
%time sector_data = get_sector(date)

Wall time: 38.9 s


In [47]:
# 업종별로 PER / PBR / 배당수익률 정보 join
sector_data2 = {k: pd.merge(v, 
                          per_pbr_divdend_data, 
                          on=['종목코드']).sort_values('PER')
                for k, v in sector_data.items()}

In [50]:
sector_data2['G1010']
# a = sector_data.items()

,종목명_x,종목코드,섹터명,종목명_y,종가,PER,PBR,배당수익률,기준일
16,대성산업,128820,WICS 에너지,대성산업,4235,3.00,0.38,0.00,20211207
4,SK디스커버리,006120,WICS 에너지,SK디스커버리,46300,3.54,0.49,2.16,20211207
26,중앙에너비스,000440,WICS 에너지,중앙에너비스,15900,3.84,1.61,3.02,20211207
25,경동인베스트,012320,WICS 에너지,경동인베스트,35100,5.05,0.20,1.42,20211207
15,미창석유,003650,WICS 에너지,미창석유,79000,6.09,0.50,2.66,20211207
23,스페코,013810,WICS 에너지,스페코,5300,8.53,1.89,0.94,20211207
6,KG케미칼,001390,WICS 에너지,KG케미칼,31400,9.35,1.07,1.27,20211207
14,SDN,099220,WICS 에너지,SDN,1855,10.54,0.99,0.00,20211207
8,한국쉘석유,002960,WICS 에너지,한국쉘석유,276500,17.11,3.66,5.06,20211207
17,극동유화,014530,WICS 에너지,극동유화,3735,17.54,0.76,3.75,20211207


In [29]:
sector_code_name = {k: ' '.join(v['섹터명'].iloc[0].split()[1:]) \
                    for k,v in sector_data2.items()}

In [31]:
# PER 평균값 데이터
per_mean = dict()

# 시장 PER 평균
per_mean['전체'] = per_pbr_divdend_data['PER'].mean()

# 업종별 PER 평균
for k,v in sector_data2.items():
    per_mean[sector_code_name[k]] = v['PER'].mean()

In [32]:
per_mean_df = pd.DataFrame(per_mean.items())
per_mean_df.columns = ['업종', '평균PER']

In [33]:
per_mean_df.sort_values('평균PER')

,업종,평균PER
20,부동산,7.780000
19,보험,9.874286
17,증권,10.298696
18,다각화된금융,16.117727
4,상업서비스와공급품,18.040526
28,유틸리티,22.092500
8,"호텔,레스토랑,레저등",23.476667
16,은행,25.687273
13,가정용품과개인용품,26.876000
25,디스플레이,29.303462


In [34]:
def pass_criteria(code):

    # 재무제표 데이터에서 원하는 조건을 만족하는 경우 True, 아니면 False 반환
    annual_data, quarter_data = get_financial_summary(code)

    try:
        # 최근 3개년 영업이익 흑자
        num_year = 3
        annual_profit = annual_data['영업이익'].tail(num_year)
        if not all(x > 0 for x in annual_profit):
            return False

        # 최근 3개년 ROE >= 8
        num_year = 3
        annual_roe = annual_data['ROE'].tail(num_year)
        if not all(x >= 8 for x in annual_roe):
            return False

        # 최근 3개년 배당수익률 >= 2
        num_year = 3
        annual_dvd = annual_data['배당수익률'].tail(num_year)
        if not all(x >= 2 for x in annual_dvd):
            return False

        # 최근 4분기 영업이익 흑자
        num_quarter = 4
        quarter_profit = quarter_data['영업이익'].tail(num_quarter)
        if not all(x > 0 for x in quarter_profit):
            return False
    except:
        # 체크하려는 값이 nan인 경우거나, 데이터가 불충분한 경우
        return False

    return True


In [36]:
# 기준값 만족하는 종목을 업종별로 스크리닝
corp_by_sector = dict()

for sector_code, sector_df in tqdm(sector_data2.items()):

    sector_name = sector_code_name[sector_code]
    cutoff_per = sector_df['PER'].quantile(0.5)
    
    # PER이 낮은 종목 -> 높은 종목 순으로 체크
    for _, x in sector_df.iterrows():

        if pass_criteria(x['종목코드']):
            corp_by_sector[sector_name] = dict(x)
            break

        # 업종 PER의 중위수보다 작은 PER을 갖는 종목만 고려한다
        elif x['PER'] >= cutoff_per:
            break

    # 적절한 종목이 없는 경우, 빈값을 채운다
    if sector_name not in corp_by_sector:
        corp_by_sector[sector_name] = {'기준일':x['기준일']}

100%|██████████| 28/28 [08:26<00:00, 18.08s/it]


In [37]:
len(corp_by_sector)

28

In [51]:
corp_by_sector_df = pd.DataFrame(corp_by_sector.values())
corp_by_sector_df['업종'] = corp_by_sector.keys()
corp_by_sector_df['업종평균PER'] = [per_mean[k] for k in corp_by_sector]
corp_by_sector_df = corp_by_sector_df[['기준일', '업종', '종목명_x','종목코드', '종가', 
                                       'PER', 'PBR', '배당수익률', '업종평균PER']]
corp_by_sector_df.fillna('', inplace=True)

In [52]:
corp_by_sector_df

,기준일,업종,종목명_x,종목코드,종가,PER,PBR,배당수익률,업종평균PER
0,20211207,에너지,,,,,,,33.358125
1,20211207,소재,애경케미칼,161000,11100.0,7.89,0.9,3.15,284.357964
2,20211207,자본재,태영건설,009410,10500.0,1.1,0.66,3.1,49.252600
3,20211207,상업서비스와공급품,고려신용정보,049720,8750.0,11.82,4.29,3.14,18.040526
4,20211207,운송,,,,,,,40.787826
5,20211207,자동차와부품,대유에이피,290120,5980.0,11.28,1.3,3.34,67.786818
6,20211207,내구소비재와의류,쿠쿠홀딩스,192400,18800.0,5.47,0.88,3.51,46.839205
7,20211207,"호텔,레스토랑,레저등",,,,,,,23.476667
8,20211207,소매(유통),,,,,,,121.488500
9,20211207,교육서비스,씨엠에스에듀,225330,7150.0,21.47,3.13,3.64,80.801667
